In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uldisvalainis/audio-emotions")

print("Path to dataset files:", path)

Path to dataset files: /usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis/audio-emotions/versions/1


In [21]:
import os
import librosa
import numpy as np
import json

# Base directory
emotions = ['Happy', 'Sad', 'Disgusted', 'Neutral', 'Angry', 'Suprised', 'Fearful']

# Base directory
base_directory = "/usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis/audio-emotions/versions/1/Emotions"

# Dictionary to store vectors
audio_vectors = {}

# Traverse and process .wav files
for file_name in os.listdir(base_directory):
    file_path = os.path.join(base_directory, file_name)
    
    if os.path.isfile(file_path) and file_name.endswith('.wav'):
        print(file_path)

# Save the vectors to a JSON file
output_path = "audio_vectors.json"
with open(output_path, "w") as json_file:
    json.dump(audio_vectors, json_file)

print(f"Audio vectors saved to {output_path}")

base_directory = "/usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis/audio-emotions/versions/1/Emotions"
def traverse_emotion(emotion, directory):
    audio_vectors = {}
    #GetMelspectogram 
    #GetVectorofWords
    #Put in pandas dataframe
    directory = directory + "/" + emotion
    # Traverse and process .wav files
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        
        if os.path.isfile(file_path) and file_name.endswith('.wav'):
            print(file_path)
traverse_emotion(emotions[2], base_directory)

Audio vectors saved to audio_vectors.json
Disgusted
/usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis/audio-emotions/versions/1/Emotions/Disgusted/03-01-07-02-02-01-07.wav
/usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis/audio-emotions/versions/1/Emotions/Disgusted/1055_IWL_DIS_XX.wav
/usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis/audio-emotions/versions/1/Emotions/Disgusted/1084_WSI_DIS_XX.wav
/usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis/audio-emotions/versions/1/Emotions/Disgusted/1079_TIE_DIS_XX.wav
/usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis/audio-emotions/versions/1/Emotions/Disgusted/YAF_nice_disgust.wav
/usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis/audio-emotions/versions/1/Emotions/Disgusted/1017_TIE_DIS_XX.wav
/usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis/audio-emotions/versions/1/Emotions/Disgusted/1064_TAI_DIS_XX.wav
/usr4/ugrad/aditya12/.cache/kagglehub/datasets/uldisvalainis

In [22]:
# import os

# # Print all files in the downloaded dataset path
# for root, dirs, files in os.walk(path):
#     print("Root Directory:", root)
#     print("Subdirectories:", dirs)
#     print("Files:", files)
